In [19]:
from pyzbar.pyzbar import decode
from PIL import Image

In [20]:
def decode_barcodes_from_dataset(dataset_path):
    """
    Декодирует штрихкоды из изображений в указанной директории.

    Args:
        dataset_path (str): Путь к директории, содержащей изображения штрихкодов.

    Returns:
        list: Список кортежей, где каждый кортеж содержит имя файла изображения и декодированный текст штрихкода.
              Возвращает пустой список, если штрихкоды не найдены или произошла ошибка.
    """
    import os
    results = []

    for filename in os.listdir(dataset_path):
        if filename.endswith(('.png', '.jpg', '.jpeg', '.gif')): # Поддерживаемые форматы изображений
            filepath = os.path.join(dataset_path, filename)
            try:
                # Открываем изображение с помощью Pillow (PIL)
                image = Image.open(filepath)

                # Декодируем штрихкоды
                barcodes = decode(image)

                if barcodes:
                    for barcode in barcodes:
                        # Получаем данные штрихкода
                        barcode_data = barcode.data.decode('utf-8')
                        results.append((filename, barcode_data))
                else:
                    print(f'Штрихкод не найден в файле: {filename}')

            except FileNotFoundError:
                print(f'Ошибка: Файл не найден: {filepath}')
            except Exception as e:
                print(f'Ошибка при обработке файла {filename}: {e}')
                

    return results

In [21]:
# Пример использования:
dataset_folder = '/Users/admin/Desktop/Vanguard/warehouse_photos&video_pixel7'
decoded_barcodes = decode_barcodes_from_dataset(dataset_folder)

if decoded_barcodes:
    for filename, barcode_data in decoded_barcodes:
        print(f'Файл: {filename}, Штрихкод: {barcode_data}')
else:
    print('Штрихкоды не обнаружены.')

Штрихкод не найден в файле: PXL_20230826_095205237.jpg
Файл: PXL_20230826_081352782.jpg, Штрихкод: B-0009-Z
Файл: PXL_20230826_081352782.jpg, Штрихкод: B-0008-Z
Файл: PXL_20230826_081352782.jpg, Штрихкод: B-0031-Z
Файл: PXL_20230826_081352782.jpg, Штрихкод: B-0030-Z
Файл: PXL_20230826_120920147.jpg, Штрихкод: B-0008-Z
Файл: PXL_20230826_081448691.jpg, Штрихкод: A-0002-Z
Файл: PXL_20230826_081448691.jpg, Штрихкод: A-0010-Z
Файл: PXL_20230826_081448691.jpg, Штрихкод: A-0005-Z
Файл: PXL_20230826_081448691.jpg, Штрихкод: B-0025-Z
Файл: PXL_20230826_081448691.jpg, Штрихкод: B-0029-Z
Файл: PXL_20230826_081448691.jpg, Штрихкод: B-0024-Z
Файл: PXL_20230826_081448691.jpg, Штрихкод: B-0007-Z
Файл: PXL_20230826_081448691.jpg, Штрихкод: B-0028-Z
Файл: PXL_20230826_081448691.jpg, Штрихкод: B-0023-Z
Файл: PXL_20230826_081448691.jpg, Штрихкод: B-0006-Z
Файл: PXL_20230826_081448691.jpg, Штрихкод: B-0027-Z
Файл: PXL_20230826_081448691.jpg, Штрихкод: B-0005-Z
Файл: PXL_20230826_081448691.jpg, Штрихкод: 

In [22]:
from pyzbar.pyzbar import decode
from PIL import Image
import os

def jaccard_index(set1, set2):
    """
    Вычисляет индекс Жаккара для двух множеств.

    Args:
        set1 (set): Первое множество.
        set2 (set): Второе множество.

    Returns:
        float: Индекс Жаккара (от 0 до 1). 1 означает полное совпадение.
    """
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return intersection / union if union else 1.0 # Обработка случая, когда оба множества пусты

def decode_barcodes_from_dataset(dataset_path, expected_data=None):
    """
    Декодирует штрихкоды из изображений в указанной директории и вычисляет индекс Жаккара.

    Args:
        dataset_path (str): Путь к директории, содержащей изображения штрихкодов.
        expected_data (dict, optional): Словарь, где ключи - имена файлов, значения - ожидаемые данные штрихкодов.
                                         Если None, метрика Жаккара не вычисляется.

    Returns:
        list: Список кортежей (имя файла, декодированный текст).
        float: Средний индекс Жаккара, если expected_data указан, иначе - None.
    """
    results = []
    jaccard_scores = []

    for filename in os.listdir(dataset_path):
        if filename.endswith(('.png', '.jpg', '.jpeg', '.gif')):
            filepath = os.path.join(dataset_path, filename)
            try:
                image = Image.open(filepath)
                barcodes = decode(image)
                decoded_texts = set() # Используем set для уникальности
                if barcodes:
                    for barcode in barcodes:
                        barcode_data = barcode.data.decode("utf-8")
                        decoded_texts.add(barcode_data)
                    results.append((filename, list(decoded_texts))) # Сохраняем список строк

                    if expected_data and filename in expected_data:
                        expected_texts = set(expected_data[filename]) if isinstance(expected_data[filename],list) else {expected_data[filename]}
                        score = jaccard_index(decoded_texts, expected_texts)
                        jaccard_scores.append(score)
                else:
                     results.append((filename, [])) # Штрихкод не найден
                     print(f'Штрихкод не найден в файле: {filename}')

            except FileNotFoundError:
                print(f'Ошибка: Файл не найден: {filepath}')
            except Exception as e:
                print(f'Ошибка при обработке файла {filename}: {e}')
                results.append((filename, []))

    avg_jaccard_score = sum(jaccard_scores) / len(jaccard_scores) if jaccard_scores else None
    return results, avg_jaccard_score

In [23]:
# Пример использования:
dataset_folder = '/Users/admin/Desktop/Vanguard/warehouse_photos&video_pixel7'
# Пример expected_data 
expected_data = {
    'PXL_20230826_095205237.jpg': ['B-0030-Z', 'B-0031-Z', 'B-0032-Z', 'B-0033-Z', 'B-0034-Z', 'B-0035-Z', 'B-0036-Z', 'B-0037-Z', 'B-0038-Z', 'B-0018-Z', 'B-0019-Z', 'B-0008-Z', 'B-0009-Z'], 
    'PXL_20230826_120920147.jpg': ['B-0008-Z'],
    'PXL_20230826_081448691.jpg': ['B-0005-Z', 'B-0025-Z', 'B-0029-Z', 'B-0024-Z', 'A-0010-Z', 'B-0006-Z', 'A-0002-Z', 'B-0027-Z', 'A-0005-Z', 'B-0028-Z', 'B-0004-Z', 'B-0023-Z', 'B-0007-Z', 'B-0026-Z'],
    'PXL_20230826_081602262.jpg': ['B-0010-Z', 'B-0011-Z', 'B-0001-Z', 'B-0012-Z', 'B-0020-Z'],
    'PXL_20230826_120915794.jpg': ['B-0038-Z', 'B-0018-Z'],
    'PXL_20230826_081428022.jpg': ['A-0021-Z'],
    'PXL_20230826_081420320.jpg': ['A-0033-Z', 'A-0017-Z', 'B-0013-Z', 'A-0016-Z', 'A-0020-Z', 'A-0018-Z', 'B-0015-Z'],
    'PXL_20230826_081420320.jpg': ['B-0031-Z', 'B-0032-Z', 'B-0009-Z', 'B-0030-Z'],
    'PXL_20230826_081420320.jpg': ['A-0014-Z', 'B-0016-Z']
}

decoded_barcodes, avg_jaccard = decode_barcodes_from_dataset(dataset_folder, expected_data)

if decoded_barcodes:
    for filename, barcode_data in decoded_barcodes:
        print(f'Файл: {filename}, Штрихкод: {barcode_data}')

    if avg_jaccard is not None:
        print(f'Средний индекс Жаккара: {avg_jaccard:.4f}')
else:
    print('Штрихкоды не обнаружены.')


Штрихкод не найден в файле: PXL_20230826_095205237.jpg
Файл: PXL_20230826_095205237.jpg, Штрихкод: []
Файл: PXL_20230826_081352782.jpg, Штрихкод: ['B-0031-Z', 'B-0009-Z', 'B-0030-Z', 'B-0008-Z']
Файл: PXL_20230826_120920147.jpg, Штрихкод: ['B-0008-Z']
Файл: PXL_20230826_081448691.jpg, Штрихкод: ['B-0005-Z', 'B-0025-Z', 'B-0029-Z', 'B-0024-Z', 'A-0010-Z', 'B-0006-Z', 'A-0002-Z', 'B-0027-Z', 'A-0005-Z', 'B-0028-Z', 'B-0004-Z', 'B-0023-Z', 'B-0007-Z', 'B-0026-Z']
Файл: PXL_20230826_081602262.jpg, Штрихкод: ['B-0010-Z', 'B-0011-Z', 'B-0001-Z', 'B-0012-Z', 'B-0020-Z']
Файл: PXL_20230826_120915794.jpg, Штрихкод: ['B-0038-Z', 'B-0018-Z']
Файл: PXL_20230826_081428022.jpg, Штрихкод: ['A-0021-Z']
Файл: PXL_20230826_081420320.jpg, Штрихкод: ['A-0033-Z', 'A-0017-Z', 'B-0013-Z', 'A-0016-Z', 'A-0020-Z', 'A-0018-Z', 'B-0015-Z']
Файл: PXL_20230826_095159075.jpg, Штрихкод: ['B-0031-Z', 'B-0032-Z', 'B-0009-Z', 'B-0030-Z']
Файл: PXL_20230826_081440099.jpg, Штрихкод: ['A-0014-Z', 'B-0016-Z']
Средний индек